In [1]:
# Import the modules
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, matthews_corrcoef, cohen_kappa_score
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

2024-06-09 20:51:38.174909: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [3]:
# Read the csv file into a pandas DataFrame
murder_target = pd.read_csv(
    Path("../Crime/murder_target.csv"), encoding='iso-8859-1'
)

# Review the DataFrame
murder_target.head()

,offense_code,offense_code_extension,offense_type_id,offense_category_id,reported_date,incident_address,geo_lon,geo_lat,district_id,precinct_id,neighborhood_id,victim_count,offense_type_name,murders
0,2999,0,criminal-mischief-other,public-disorder,2/10/2022 3:16:00 AM,1107 N SANTA FE DR,-104.998910,39.733957,1,123.0,lincoln-park,1.0,Criminal mischief - other,0
1,2999,0,criminal-mischief-other,public-disorder,7/8/2021 12:55:00 AM,815 16TH ST,-104.993342,39.746248,6,611.0,cbd,1.0,Criminal mischief - other,0
2,2999,0,criminal-mischief-other,public-disorder,10/29/2020 4:31:00 AM,4745 N FEDERAL BLVD,-105.025520,39.782888,1,111.0,berkeley,1.0,Criminal mischief - other,0
3,2999,0,criminal-mischief-other,public-disorder,9/7/2018 9:58:00 AM,65 S FEDERAL BLVD,-105.025330,39.715357,4,411.0,barnum,1.0,Criminal mischief - other,0
4,2999,0,criminal-mischief-other,public-disorder,5/13/2020 10:00:00 AM,12295 E ALBROOK DR,-104.845074,39.783082,5,521.0,montbello,1.0,Criminal mischief - other,0


In [ ]:
murder_target.columns

In [ ]:
murder_target.info()

In [ ]:
# Handle missing values
murder_target = murder_target.dropna(subset=['district_id', 'murders'])  # Drop rows where district_id or murders is NaN

# Convert 'district_id' to numerical if necessary
murder_target['district_id'] = pd.to_numeric(murder_target['district_id'], errors='coerce')

In [ ]:
# Drop rows where 'district_id' could not be converted to numeric
murder_target = murder_target.dropna(subset=['district_id'])


In [ ]:
# Convert 'district_id' to integer
murder_target['district_id'] = murder_target['district_id'].astype(int)

# Ensure 'murders' is integer
murder_target['murders'] = murder_target['murders'].astype(int)

In [ ]:
#Step 3: Split the Data
X = murder_target[['district_id']]  # Features
y = murder_target['murders']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train a Model
model = LogisticRegression(max_iter=1000)  # Use Logistic Regression for binary classification
model.fit(X_train, y_train)


In [ ]:
# Step 5: Evaluate the Model
y_pred = model.predict(X_test)

In [ ]:
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

In [ ]:
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

In [ ]:
# Classification Report
class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)

In [ ]:
#Attempt #2

In [ ]:
# Select features and target
X = murder_target[['district_id']]
y = murder_target['murders']

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Handle imbalance using SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Train the model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_res, y_train_res)

In [ ]:
# Make predictions
y_pred = rf_model.predict(X_test)

In [ ]:
# Generate classification report
print(classification_report(y_test, y_pred, zero_division=1))

# Display confusion matrix
print(confusion_matrix(y_test, y_pred))

In [ ]:
#Attempt Number 3

In [ ]:
# Select features and target
X = murder_target[['district_id', 'precinct_id', 'neighborhood_id', 'victim_count']]
y = murder_target['murders']

In [ ]:
# Ensure categorical columns are strings
X.loc[:, 'district_id'] = X['district_id'].astype(str)
X.loc[:, 'neighborhood_id'] = X['neighborhood_id'].astype(str)

# Ensure numerical columns are numeric
X.loc[:, 'precinct_id'] = pd.to_numeric(X['precinct_id'], errors='coerce')
X.loc[:, 'victim_count'] = pd.to_numeric(X['victim_count'], errors='coerce')

In [ ]:
# Handle missing values
X = X.fillna(-1)

In [ ]:
# One-hot encode categorical features
categorical_features = ['district_id', 'neighborhood_id']
numerical_features = ['precinct_id', 'victim_count']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

In [ ]:
# Create a pipeline for preprocessing
preprocessing_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

In [ ]:
# Preprocess the data
X_preprocessed = preprocessing_pipeline.fit_transform(X)

# Apply SMOTE to the preprocessed data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_preprocessed, y)

# Split the resampled data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier(random_state=42, class_weight={0: 1, 1: 10})
model.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

In [ ]:
# Generate classification report
print(classification_report(y_test, y_pred, zero_division=1))

# Display confusion matrix
print(confusion_matrix(y_test, y_pred))


In [ ]:
#Attempt #4

In [ ]:
# Select features and target
X = murder_target[['district_id', 'precinct_id', 'neighborhood_id', 'victim_count']]
y = murder_target['murders']

In [ ]:
# Ensure categorical columns are strings
X['district_id'] = X['district_id'].astype(str)
X['neighborhood_id'] = X['neighborhood_id'].astype(str)

# Ensure numerical columns are numeric
X['precinct_id'] = pd.to_numeric(X['precinct_id'], errors='coerce')
X['victim_count'] = pd.to_numeric(X['victim_count'], errors='coerce')

In [ ]:
# Handle missing values
X = X.fillna(-1)

# One-hot encode categorical features
categorical_features = ['district_id', 'neighborhood_id']
numerical_features = ['precinct_id', 'victim_count']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

In [ ]:
# Create a pipeline for preprocessing
preprocessing_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Preprocess the data
X_preprocessed = preprocessing_pipeline.fit_transform(X)

# Apply SMOTETomek to the preprocessed data
smotetomek = SMOTETomek(random_state=42)
X_resampled, y_resampled = smotetomek.fit_resample(X_preprocessed, y)

In [ ]:
# Split the resampled data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Set up the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': [{0: 1, 1: 10}, 'balanced']
}

In [ ]:
# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=3,
                           n_jobs=-1,
                           scoring='f1')

# Fit the model
grid_search.fit(X_train, y_train)

# Make predictions
y_pred = grid_search.best_estimator_.predict(X_test)

In [ ]:
# Generate classification report
print(classification_report(y_test, y_pred, zero_division=1))

# Display confusion matrix
print(confusion_matrix(y_test, y_pred))

In [ ]:
#Attempt #5   Using the kappa & matthews method 

In [ ]:
# Select features and target
X = murder_target[['district_id', 'precinct_id', 'neighborhood_id', 'victim_count']].copy()
y = murder_target['murders']

In [ ]:
# Ensure categorical columns are strings
X.loc[:, 'district_id'] = X['district_id'].astype(str)
X.loc[:, 'neighborhood_id'] = X['neighborhood_id'].astype(str)

# Ensure numerical columns are numeric
X.loc[:, 'precinct_id'] = pd.to_numeric(X['precinct_id'], errors='coerce')
X.loc[:, 'victim_count'] = pd.to_numeric(X['victim_count'], errors='coerce')

# Handle missing values
X = X.fillna(-1)

In [ ]:
# One-hot encode categorical features
categorical_features = ['district_id', 'neighborhood_id']
numerical_features = ['precinct_id', 'victim_count']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

In [ ]:
# Create a pipeline for preprocessing
preprocessing_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Sample a subset of data (e.g., 10%)
X_sampled = X.sample(frac=0.1, random_state=42)
y_sampled = y[X_sampled.index]

In [ ]:
# Check memory usage
print(f"Original data size: {X.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
print(f"Sampled data size: {X_sampled.memory_usage(deep=True).sum() / (1024**2):.2f} MB")

# Preprocess the sampled data
X_preprocessed_sampled = preprocessing_pipeline.fit_transform(X_sampled)
print("Preprocessing done")

In [ ]:
# Adjust SMOTETomek parameters for efficiency
smotetomek = SMOTETomek(random_state=42, n_jobs=-1, sampling_strategy='auto')

# Apply SMOTETomek to the sampled data
X_resampled_sampled, y_resampled_sampled = smotetomek.fit_resample(X_preprocessed_sampled, y_sampled)
print("Resampling done with sampled data")

In [ ]:
# Split the resampled sampled data
X_train, X_test, y_train, y_test = train_test_split(X_resampled_sampled, y_resampled_sampled, test_size=0.2, random_state=42)
print("Train-test split done")

In [ ]:
# Set up a simplified parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100],
    'max_depth': [10],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'class_weight': ['balanced']
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=3,
                           n_jobs=-1,
                           scoring='f1')

# Fit the model
grid_search.fit(X_train, y_train)
print("Grid search done")

In [ ]:
# Make predictions
y_pred = grid_search.best_estimator_.predict(X_test)

In [ ]:
# Generate classification report
print(classification_report(y_test, y_pred, zero_division=1))

# Display confusion matrix
print(confusion_matrix(y_test, y_pred))

# Calculate Matthews Correlation Coefficient (MCC)
mcc = matthews_corrcoef(y_test, y_pred)
print(f"Matthews Correlation Coefficient (MCC): {mcc}")

# Calculate Cohen's Kappa
kappa = cohen_kappa_score(y_test, y_pred)
print(f"Cohen's Kappa: {kappa}")